In [22]:
from cassandra.cluster import Cluster, PlainTextAuthProvider
from cassandra.cluster import (
    NoHostAvailable,
    OperationTimedOut
)
import os, sys
import pandas as pd, numpy as np
import json

CASS_HOST = os.getenv('CASS_HOST') or 'localhost'
CASS_USER = os.getenv('CASS_USER') or 'cassandra'
CASS_PASS = os.getenv('CASS_PASS') or 'changeme'

In [23]:
weps = pd.read_csv('../../data/weapons.csv')
arm = pd.read_csv('../../data/armor.csv')
spell = pd.read_csv('../../data/parsed_spells2.csv')
class_data = pd.read_csv('../../data/class_by_level.csv')
skills = pd.read_csv('../../data/skills.csv')

In [16]:
weps.columns

Index(['Name', 'type', 'cost', 'cost_unit', 'dmg', 'props', 'normal_range',
       'long_range', 'mod', 'wgt'],
      dtype='object')

In [17]:
arm.columns

Index(['weName', 'minSTR', 'stealth_disadv', 'Weight', 'type', 'baseAC',
       'modAC', 'cost_amt', 'cost_unit', 'Don', 'Doff'],
      dtype='object')

In [18]:
spell.columns

Index(['Lvl', 'Name', 'Save', 'DmgType', 'HigherLvlIncrease', 'Upcast',
       'hard_mod', 'mult', 'upcast', 'dmg', 'addspellmod', 'time', 'range',
       'components', 'duration', 'classes', 'conc', 'desc', 'save_data',
       'save_success'],
      dtype='object')

In [24]:
auth_provider = PlainTextAuthProvider(
    username = CASS_USER,
    password = CASS_PASS
)
print(CASS_HOST,CASS_USER,CASS_PASS)
cluster = Cluster(
    [CASS_HOST],
    auth_provider=auth_provider
)
session = cluster.connect()

localhost cassandra changeme


In [24]:
session.execute('use dndio;')
for i, row in arm.iterrows():
    # print(row)
    query = """ 
INSERT INTO armor (
    id,
    baseac,
    cost_amt,
    cost_unit,
    doff,
    don,
    minstr,
    modac,
    name,
    stealth_disadv,
    type,
    weight
) VALUES (
    uuid(),
    {},
    {},
    '{}',
    '{}',
    '{}',
    {},
    '{}',
    '{}',
    {},
    '{}',
    {}
);
""".format(
    row['baseAC'],row['cost_amt'],row['cost_unit'],row['Doff'],row['Don'],
    row['minSTR'],row['modAC'],row['Name'],str(row['stealth_disadv']).lower(),row['type'],row['Weight']
)
    print(query)
    session.execute(query)
    # print(query)
    # ['Name', 'minSTR', 'stealth_disadv', 'Weight', 'type', 'baseAC', 'modAC',
    #    'cost_amt', 'cost_unit', 'Don', 'Doff']

KeyError: 'Name'

In [25]:
weps.columns
#columns in db needed...name text, cost_unit text, cost_amt smallint, properties(list), type text, norm_range smallint, long_range smallint, mod (list<text>)
#dmg list<frozen<map<text,frozen<map<smallint,smallint>>>>>>, 
#dmg2hnd list<frozen<map<text,frozen<map<smallint,smallint>>>>>>, 
#weight float?
# 

Index(['Name', 'type', 'cost', 'cost_unit', 'dmg', 'props', 'normal_range',
       'long_range', 'mod', 'wgt'],
      dtype='object')

In [76]:
weps.drop(columns=[
    'Cost','Damage','Weight','Properties','dmg_die','dmg_type','dmg_die_qty','dmg_die_size','2hnd_die_qty','2hnd_die_size','weight'
])


KeyError: "['Cost', 'Damage', 'Weight', 'Properties', 'dmg_die', 'dmg_type', 'dmg_die_qty', 'dmg_die_size', '2hnd_die_qty', '2hnd_die_size', 'weight'] not found in axis"

create table weapons (
    id uuid,
    name text,
    type text,
    cost_amt smallint,
    cost_unit text,
    dmg list<frozen<map<text,frozen<map<smallint,smallint>>>>>,
    dmg2hnd list<frozen<map<text,frozen<map<smallint,smallint>>>>>,
    mod list<text>,
    norm_range smallint,
    long_range smallint,
    props list<text>,
    weight double,
    PRIMARY KEY (id)
);

In [77]:
weps['props']
weps['props'] = weps.apply(
    lambda row: ''.join([ row['props'][i].lower() for i in range(len(row['props']))]),
    axis=1
)
display(weps)


,Name,type,cost,cost_unit,main_type,dmg,props,normal_range,long_range,mod,wgt
0,Club,Melee,1,sp,simple,{'1hnd': {'bludgeoning': {1: 4}}},['light'],5,5,['STR'],2.00
1,Dagger,Melee,2,gp,simple,{'1hnd': {'piercing': {1: 4}}},"['finesse', 'light', 'thrown (20/60)']",20,60,"['STR', 'DEX']",1.00
2,Greatclub,Melee,2,sp,simple,{'1hnd': {'bludgeoning': {1: 8}}},['two-handed'],5,5,['STR'],10.00
3,Handaxe,Melee,5,gp,simple,{'1hnd': {'slashing': {1: 6}}},"['light', 'thrown (20/60)']",20,60,['STR'],2.00
4,Javelin,Melee,5,sp,simple,{'1hnd': {'piercing': {1: 6}}},['thrown (30/120)'],5,5,['STR'],2.00
5,Light hammer,Melee,2,gp,simple,{'1hnd': {'bludgeoning': {1: 4}}},"['light', 'thrown (20/60)']",20,60,['STR'],2.00
6,Mace,Melee,5,gp,simple,{'1hnd': {'bludgeoning': {1: 6}}},[],5,5,['STR'],4.00
7,Quarterstaff,Melee,2,sp,simple,"{'1hnd': {'bludgeoning': {1: 6}}, '2hnd': {'bl...",['versatile (1d8)'],5,5,['STR'],4.00
8,Sickle,Melee,1,gp,simple,{'1hnd': {'slashing': {1: 4}}},['light'],5,5,['STR'],2.00
9,Spear,Melee,1,gp,simple,"{'1hnd': {'piercing': {1: 6}}, '2hnd': {'pierc...","['thrown (20/60)', 'versatile (1d8)']",5,5,['STR'],3.00


In [84]:
weps.drop(columns=[
    'Cost','Damage','Weight','Properties','dmg_die','dmg_type','dmg_die_qty','dmg_die_size','2hnd_die_qty','2hnd_die_size','weight'
],inplace=True)
weps


,Name,type,cost,cost_unit,dmg,props,normal_range,long_range,mod,2hnd_dmg,wgt
0,Club,Melee,1,sp,{'bludgeoning': {'1': '4'}},['light'],5,5,['STR'],{'bludgeoning': {'1': '4'}},2.00
1,Dagger,Melee,2,gp,{'piercing': {'1': '4'}},"['finesse', 'light', 'thrown (20/60)']",20,60,"['STR', 'DEX']",{'piercing': {'1': '4'}},1.00
2,Greatclub,Melee,2,sp,{'bludgeoning': {'1': '8'}},['two-handed'],5,5,['STR'],{'bludgeoning': {'1': '8'}},10.00
3,Handaxe,Melee,5,gp,{'slashing': {'1': '6'}},"['light', 'thrown (20/60)']",20,60,['STR'],{'slashing': {'1': '6'}},2.00
4,Javelin,Melee,5,sp,{'piercing': {'1': '6'}},['thrown (30/120)'],5,5,['STR'],{'piercing': {'1': '6'}},2.00
5,Light hammer,Melee,2,gp,{'bludgeoning': {'1': '4'}},"['light', 'thrown (20/60)']",20,60,['STR'],{'bludgeoning': {'1': '4'}},2.00
6,Mace,Melee,5,gp,{'bludgeoning': {'1': '6'}},[],5,5,['STR'],{'bludgeoning': {'1': '6'}},4.00
7,Quarterstaff,Melee,2,sp,{'bludgeoning': {'1': '6'}},['versatile (1d8)'],5,5,['STR'],{'bludgeoning': {'1': '8'}},4.00
8,Sickle,Melee,1,gp,{'slashing': {'1': '4'}},['light'],5,5,['STR'],{'slashing': {'1': '4'}},2.00
9,Spear,Melee,1,gp,{'piercing': {'1': '6'}},"['thrown (20/60)', 'versatile (1d8)']",5,5,['STR'],{'piercing': {'1': '8'}},3.00


In [78]:
import json
for i,row in weps.iterrows():
    try:
        d1 = row['dmg']
        d1=d1.replace('\'','"')
        
        d1 = json.loads(d1)
        for k,v in d1[list(d1.keys())[0]].items():
            d1[list(d1.keys())[0]] = {int(k):int(v)}
        print(d1)
    except Exception as e:
        d1 = {}
        print(e)
    print(d1)
    weps.at[i,'dmg'] = d1.copy()

    try:
        d1 = row['2hnd_dmg']
        d1=d1.replace('\'','"')
        d1 = json.loads(d1)
        for k,v in d1[list(d1.keys())[0]].items():
            d1[list(d1.keys())[0]] = {int(k):int(v)}
        print(d1)
    except Exception as e:
        d1 = {}
        print(e)
    weps.at[i,'2hnd_dmg'] = d1.copy()
    print(d1)
weps

Expecting property name enclosed in double quotes: line 1 column 27 (char 26)
{}
'2hnd_dmg'


ValueError: Incompatible indexer with Series

In [79]:
weps.columns

Index(['Name', 'type', 'cost', 'cost_unit', 'main_type', 'dmg', 'props',
       'normal_range', 'long_range', 'mod', 'wgt', '2hnd_dmg'],
      dtype='object')

In [86]:
session.execute('use dndio;')
for i, row in weps.iterrows():
    # print(row)
    query = """ 
INSERT INTO weapons (
    id,
    name,
    type,
    cost_amt,
    cost_unit,
    dmg,
    dmg2hnd,
    mod,
    norm_range,
    long_range,
    props,
    weight
) VALUES (
    uuid(),
    '{}',
    '{}',
    {},
    '{}',
    [{}],
    [{}],
    {},
    {},
    {},
    {},
    {}
);
""".format(
    row['Name'],row['type'],row['cost'],row['cost_unit'],
    row['dmg'],row['2hnd_dmg'],row['mod'],row['normal_range'],
    row['long_range'],row['props'],row['wgt']
)
    print(query)
    session.execute(query)
    # ['Name', 'type', 'cost', 'cost_unit', 'dmg', 'props', 'normal_range',
    #    'long_range', 'mod', '2hnd_dmg', 'wgt'],

 
INSERT INTO weapons (
    id,
    name,
    type,
    cost_amt,
    cost_unit,
    dmg,
    dmg2hnd,
    mod,
    norm_range,
    long_range,
    props,
    weight
) VALUES (
    uuid(),
    'Club',
    'Melee',
    1,
    'sp',
    [{'bludgeoning': {1: 4}}],
    [{'bludgeoning': {1: 4}}],
    ['STR'],
    5,
    5,
    ['light'],
    2.0
);

 
INSERT INTO weapons (
    id,
    name,
    type,
    cost_amt,
    cost_unit,
    dmg,
    dmg2hnd,
    mod,
    norm_range,
    long_range,
    props,
    weight
) VALUES (
    uuid(),
    'Dagger',
    'Melee',
    2,
    'gp',
    [{'piercing': {1: 4}}],
    [{'piercing': {1: 4}}],
    ['STR', 'DEX'],
    20,
    60,
    ['finesse', 'light', 'thrown (20/60)'],
    1.0
);

 
INSERT INTO weapons (
    id,
    name,
    type,
    cost_amt,
    cost_unit,
    dmg,
    dmg2hnd,
    mod,
    norm_range,
    long_range,
    props,
    weight
) VALUES (
    uuid(),
    'Greatclub',
    'Melee',
    2,
    'sp',
    [{'bludgeoning': {1: 8}}],
    [

In [20]:
class_levels = pd.read_csv('../../data/class_by_level.csv')
class_levels

,level,prof_bonus,features,class,class_specific,spellslots
0,1,2,"['Rage', ' Unarmored Defense']",Barbarian,"{'rages': 2, 'rage_dmg': 2}",NaN
1,2,2,"['Reckless Attack', ' Danger Sense']",Barbarian,"{'rages': 2, 'rage_dmg': 2}",NaN
2,3,2,[],Barbarian,"{'rages': 3, 'rage_dmg': 2}",NaN
3,4,2,['Ability Score Improvement'],Barbarian,"{'rages': 3, 'rage_dmg': 2}",NaN
4,5,3,[],Barbarian,"{'rages': 3, 'rage_dmg': 2}",NaN
...,...,...,...,...,...,...
255,16,5,['Ability Score Improvement'],Artificer,"{'infusions_known': 10, 'infused_items': 5, 'c...","{1: 4, 2: 3, 3: 3, 4: 2, 5: 0}"
256,17,6,['-'],Artificer,"{'infusions_known': 10, 'infused_items': 5, 'c...","{1: 4, 2: 3, 3: 3, 4: 3, 5: 1}"
257,18,6,['Magic Item Master'],Artificer,"{'infusions_known': 12, 'infused_items': 6, 'c...","{1: 4, 2: 3, 3: 3, 4: 3, 5: 1}"
258,19,6,['Ability Score Improvement'],Artificer,"{'infusions_known': 12, 'infused_items': 6, 'c...","{1: 4, 2: 3, 3: 3, 4: 3, 5: 2}"


In [21]:
for i, row in class_levels.iterrows():
    if isinstance(row['spellslots'],float):
        sp = {}
    else:
        sp = row['spellslots']
    if isinstance(row['class_specific'],float): # is None:
        cs = {}
    else:
        cs = row['class_specific']
    if isinstance(row['features'],float):
        feat = []
    else:
        feat = row['features']
        # for f in row['features']:

        #     if '\'' in f
        #         f
    query = """ 
INSERT INTO classes (
    class_id,
    level,
    prof_bonus,
    features,
    class,
    class_specific,
    spellslots
) VALUES (
    '{}-{}',
    {},
    {},
    {},
    '{}',
    {},
    {}
)
""".format(
    row['class'].lower(),row['level'],row['level'],row['prof_bonus'],feat,row['class'],cs,sp
)
    try:
        session.execute(query)
    except:
        print("couldnt execute: \n{}".format(query))

couldnt execute: 
 
INSERT INTO classes (
    class_id,
    level,
    prof_bonus,
    features,
    class,
    class_specific,
    spellslots
) VALUES (
    'ranger-8',
    8,
    3,
    ['Ability Score Improvement', " Land's Stride", '  Martial Versatility (Optional)'],
    'Ranger',
    {'spells_known': 5},
    {1: 4, 2: 3, 3: 0, 4: 0, 5: 0}
)

couldnt execute: 
 
INSERT INTO classes (
    class_id,
    level,
    prof_bonus,
    features,
    class,
    class_specific,
    spellslots
) VALUES (
    'ranger-10',
    10,
    4,
    ['Natural Explorer Improvement', ' Hide in Plain Sight', '  Deft Explorer Feature (Optional)  ', "  Nature's Veil (Optional)"],
    'Ranger',
    {'spells_known': 6},
    {1: 4, 2: 3, 3: 2, 4: 0, 5: 0}
)

couldnt execute: 
 
INSERT INTO classes (
    class_id,
    level,
    prof_bonus,
    features,
    class,
    class_specific,
    spellslots
) VALUES (
    'rogue-1',
    1,
    2,
    ['Expertise', ' Sneak Attack', " Thieves' Cant"],
    'Rogue',
    {

## Updated Insertion Code Starts Here

In [25]:
spells = pd.read_csv('../../data/parsed_spells2.csv')


In [26]:
spells.head(10)

,Lvl,Name,Save,DmgType,HigherLvlIncrease,Upcast,hard_mod,mult,upcast,dmg,addspellmod,time,range,components,duration,classes,conc,desc,save_data,save_success
0,0,Acid Splash,DEX,Acid,True,NaN,0,1,NaN,"{1: {'Acid': [{1: 6}]}, 5: {'Acid': [{2: 6}]},...",False,1 action,60 feet,"V, S",Instantaneous,"['Artificer', 'Sorcerer', 'Wizard']",False,Source: Player's Handbook\r\n Conjuration cant...,or take 1d6 acid damage.,no effect
1,0,Booming Blade,NaN,Thunder,True,NaN,0,1,NaN,{1: {'Thunder': [{1: 8}]}},False,1 action,Self (5-foot radius),"S, M (a melee weapon worth at least 1 sp)",1 round,"['Artificer', 'Sorcerer', 'Warlock', 'Wizard']",False,Source: Tasha's Cauldron of Everything\r\n Evo...,NaN,NaN
2,0,Chill Touch,NaN,Necrotic,True,NaN,0,1,NaN,"{1: {'Necrotic': [{1: 8}]}, 5: {'Necrotic': [{...",False,1 action,120 feet,"V, S",1 round,"['Sorcerer', 'Warlock', 'Wizard']",False,Source: Player's Handbook\r\n Necromancy cantr...,NaN,NaN
3,0,Create Bonfire,DEX,Fire,True,NaN,0,1,NaN,"{1: {'Fire': [{1: 8}]}, 5: {'Fire': [{2: 8}]},...",False,1 action,60 feet,"V, S","Concentration, up to 1 minute","['Artificer', 'Druid', 'Sorcerer', 'Warlock', ...",True,Source: Xanathar's Guide to Everything\r\n Con...,or take 1d8 fire damage. A creature must also...,no effect
4,0,Eldritch Blast,NaN,Force,True,NaN,0,1,NaN,{1: {'Force': [{1: 10}]}},False,1 action,120 feet,"V, S",Instantaneous,['Warlock'],False,Source: Player's Handbook\r\n Evocation cantri...,NaN,NaN
5,0,Fire Bolt,NaN,Fire,True,NaN,0,1,NaN,{1: {'Fire': [{1: 10}]}},False,1 action,120 feet,"V, S",Instantaneous,"['Artificer', 'Sorcerer', 'Wizard']",False,Source: Player's Handbook\r\n Evocation cantri...,NaN,NaN
6,0,Frostbite,CON,Cold,True,NaN,0,1,NaN,"{1: {'Cold': [{1: 6}]}, 5: {'Cold': [{2: 6}]},...",False,1 action,60 feet,"V, S",Instantaneous,"['Druid', 'Sorcerer', 'Warlock', 'Wizard', 'Ar...",False,Source: Xanathar's Guide to Everything\r\n Evo...,". On a failed save, the target takes 1d6 cold ...",no effect
7,0,Green-Flame Blade,NaN,Fire,True,NaN,0,1,NaN,{1: {'Fire': [{1: 8}]}},False,1 action,Self (5-foot radius),"S, M (a melee weapon worth at least 1 sp)",Instantaneous,"['Artificer', 'Sorcerer', 'Warlock', 'Wizard']",False,Source: Tasha's Cauldron of Everything/Sword C...,NaN,NaN
8,0,Infestation,CON,Poison,True,NaN,0,1,NaN,"{1: {'Poison': [{1: 6}]}, 5: {'Poison': [{2: 6...",False,1 action,30 feet,"V, S, M (a living flea)",Instantaneous,"['Druid', 'Sorcerer', 'Warlock', 'Wizard']",False,Source: Xanathar's Guide to Everything\r\n Con...,", or it takes 1d6 poison damage and moves 5 fe...",no effect
9,0,Lightning Lure,STR,Lightning,True,NaN,0,1,NaN,"{1: {'Lightning': [{1: 8}]}, 5: {'Lightning': ...",False,1 action,Self (15-foot radius),V,Instantaneous,"['Artificer', 'Sorcerer', 'Warlock', 'Wizard']",False,Source: Tasha's Cauldron of Everything\r\n Evo...,or be pulled up to 10 feet in a straight line...,no effect


In [27]:
spells.columns

Index(['Lvl', 'Name', 'Save', 'DmgType', 'HigherLvlIncrease', 'Upcast',
       'hard_mod', 'mult', 'upcast', 'dmg', 'addspellmod', 'time', 'range',
       'components', 'duration', 'classes', 'conc', 'desc', 'save_data',
       'save_success'],
      dtype='object')

In [28]:
spells.drop(columns=['DmgType','HigherLvlIncrease','Upcast'],axis=1,inplace=True)
spells

,Lvl,Name,Save,hard_mod,mult,upcast,dmg,addspellmod,time,range,components,duration,classes,conc,desc,save_data,save_success
0,0,Acid Splash,DEX,0,1,NaN,"{1: {'Acid': [{1: 6}]}, 5: {'Acid': [{2: 6}]},...",False,1 action,60 feet,"V, S",Instantaneous,"['Artificer', 'Sorcerer', 'Wizard']",False,Source: Player's Handbook\r\n Conjuration cant...,or take 1d6 acid damage.,no effect
1,0,Booming Blade,NaN,0,1,NaN,{1: {'Thunder': [{1: 8}]}},False,1 action,Self (5-foot radius),"S, M (a melee weapon worth at least 1 sp)",1 round,"['Artificer', 'Sorcerer', 'Warlock', 'Wizard']",False,Source: Tasha's Cauldron of Everything\r\n Evo...,NaN,NaN
2,0,Chill Touch,NaN,0,1,NaN,"{1: {'Necrotic': [{1: 8}]}, 5: {'Necrotic': [{...",False,1 action,120 feet,"V, S",1 round,"['Sorcerer', 'Warlock', 'Wizard']",False,Source: Player's Handbook\r\n Necromancy cantr...,NaN,NaN
3,0,Create Bonfire,DEX,0,1,NaN,"{1: {'Fire': [{1: 8}]}, 5: {'Fire': [{2: 8}]},...",False,1 action,60 feet,"V, S","Concentration, up to 1 minute","['Artificer', 'Druid', 'Sorcerer', 'Warlock', ...",True,Source: Xanathar's Guide to Everything\r\n Con...,or take 1d8 fire damage. A creature must also...,no effect
4,0,Eldritch Blast,NaN,0,1,NaN,{1: {'Force': [{1: 10}]}},False,1 action,120 feet,"V, S",Instantaneous,['Warlock'],False,Source: Player's Handbook\r\n Evocation cantri...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,9,Psychic Scream,INT,0,1,NaN,{1: {'Psychic': [{14: 6}]}},False,1 action,90 feet,S,Instantaneous,"['Bard', 'Sorcerer', 'Warlock', 'Wizard']",False,Source: Xanathar's Guide to Everything\r\n 9th...,". On a failed save, a target takes 14d6 psychi...",half
172,9,Ravenous Void,STR,0,1,NaN,{1: {'Force': [{5: 10}]}},False,1 action,"1,000 feet","V, S, M (a small, nine-pointed star made of i...","Concentration, up to 1 minute",['Wizard'],True,Source: Explorer's Guide to Wildemount\r\n 9th...,or be pulled straight toward the sphere's cen...,no effect
173,9,Storm of Vengeance,CON,0,1,NaN,"{1: {'Acid': [{1: 6}], 'Bludgeoning': [{2: 6}]...",False,1 action,Sight,"V, S","Concentration, up to 1 minute",['Druid'],True,Source: Player's Handbook\r\n 9th-level conjur...,". On a failed save, a creature takes 2d6 thund...",no effect
174,9,Time Ravage,CON,0,1,NaN,{1: {'Necrotic': [{10: 12}]}},False,1 action,90 feet,"V, S, M (an hourglass filled with diamond dus...",Instantaneous,['Wizard'],False,Source: Explorer's Guide to Wildemount\r\n 9th...,", taking 10d12 necrotic damage on a failed sav...",half


In [32]:
import re
session.execute('use dndio;')
failed = []
for i, row in spells.iterrows():
    query = """insert into spells (level,name,save,hardmod,mult,upcast,dmg,addspellmod,time,range,components,duration,char_classes,conc,save_data,save_success) 
VALUES ({},'{}','{}',{},{},{},{},{},'{}','{}','{}','{}',{},{},'{}',
"""
    if isinstance(row['upcast'],float):# is None:
        uc = {}
    else:
        uc = row['upcast']
    if isinstance(row['dmg'],float): # is None:
        dmg = {}
    else:
        dmg=row['dmg']
    if '\'' in row['Name']:
        name = re.sub('\'','\'\'',row['Name'])
    else:
        name = row['Name']
    if isinstance(row['classes'],float):
        classes = []
    else:
        classes = [c.lower() for c in json.loads(row['classes'].replace("'",'"'))]
        print(classes)
        # classes = row['classes']
        
    conc = row['conc']=='True'
    if isinstance(row['desc'],str) and '\'' in row['desc']:
        desc= re.sub('\'','\'\'',row['desc'])
    else:
        desc = str(row['desc'])

    if isinstance(row['save_data'],str) and '\'' in row['save_data']:
        save_data = re.sub('\'','\'\'',row['save_data'])
    else:
        save_data=str(row['save_data'])

    t=query.format(
        row['Lvl'],name.lower(),row['Save'],row['hard_mod'],row['mult'],uc,dmg,row['addspellmod'],
        row['time'],row['range'],row['components'],row['duration'],classes,conc,save_data,row['save_success']
    )
    #print(t)
    t+="'"+desc + "');"
    #print(t)
    try:
        session.execute(t)
    except: 
        print("Couldn't insert {}".format(row['Name']))
        failed.append(row)
        pass

pd.DataFrame(failed)

['artificer', 'sorcerer', 'wizard']
['artificer', 'sorcerer', 'warlock', 'wizard']
['sorcerer', 'warlock', 'wizard']
['artificer', 'druid', 'sorcerer', 'warlock', 'wizard']
['warlock']
['artificer', 'sorcerer', 'wizard']
['druid', 'sorcerer', 'warlock', 'wizard', 'artificer']
['artificer', 'sorcerer', 'warlock', 'wizard']
['druid', 'sorcerer', 'warlock', 'wizard']
['artificer', 'sorcerer', 'warlock', 'wizard']
['druid', 'warlock', 'artificer']
['sorcerer', 'warlock', 'wizard']
['artificer', 'druid', 'sorcerer', 'warlock', 'wizard']
['druid']
['druid']
['artificer', 'sorcerer', 'wizard']
['cleric']
['wizard']
['druid']
['artificer', 'sorcerer', 'wizard']
['artificer', 'sorcerer', 'warlock', 'wizard']
['artificer', 'druid']
['bard', 'druid', 'sorcerer', 'warlock', 'wizard', 'artificer']
['cleric', 'warlock', 'wizard']
['bard']
['cleric']
['artificer', 'druid', 'ranger', 'sorcerer', 'wizard']
['warlock']
['warlock']
['sorcerer', 'wizard']
['sorcerer', 'wizard', 'artificer']
['sorcerer']
[

,Lvl,Name,Save,hard_mod,mult,upcast,dmg,addspellmod,time,range,components,duration,classes,conc,desc,save_data,save_success
117,5,Bigby's Hand,NaN,0,1,"{6: {'Force': [{2: 8}]}, 5: {'Bludgeoning': [{...","{1: {'Bludgeoning': [{2: 6}], 'Force': [{4: 8}]}}",True,1 action,120 feet,"V, S, M (an eggshell and a snakeskin glove)","Concentration, up to 1 minute","['Artificer', 'Sorcerer', 'Wizard']",True,Source: Player's Handbook\r\n 5th-level evocat...,NaN,NaN
149,6,Wall of Thorns,DEX,0,1,{7: {1: 8}},"{1: {'Piercing': {7: 8}, 'Slashing': {7: 8}}}",False,1 action,120 feet,"V, S, M (a handful of thorns)","Concentration, up to 10 minutes",['Druid'],True,Source: Player's Handbook\r\n 6th-level conjur...,". On a failed save, a creature takes 7d8 pierc...",half
160,8,Dark Star,CON,0,1,NaN,{1: {'Force': [{8: 10}]}},False,1 action,150 feet,"V, S, M (a shard of onyx and a drop of the ca...","Concentration, up to 1 minute",['Wizard'],True,Source: Explorer's Guide to Wildemount\r\n 8th...,. The creature takes 8d10 force damage on a fa...,half


In [118]:
weps = pd.read_csv('../../data/weapons.csv')
weps.columns

Index(['Name', 'type', 'cost', 'cost_unit', 'main_type', 'dmg', 'props',
       'normal_range', 'long_range', 'mod', 'wgt'],
      dtype='object')

In [119]:
display(weps.head(),weps.columns)

,Name,type,cost,cost_unit,main_type,dmg,props,normal_range,long_range,mod,wgt
0,club,Melee,1,sp,simple,{'1hnd': {'bludgeoning': {1: 4}}},['Light'],5,5,['STR'],2.0
1,dagger,Melee,2,gp,simple,{'1hnd': {'piercing': {1: 4}}},"['Finesse', 'light', 'thrown (20/60)']",20,60,"['STR', 'DEX']",1.0
2,greatclub,Melee,2,sp,simple,{'1hnd': {'bludgeoning': {1: 8}}},['Two-handed'],5,5,['STR'],10.0
3,handaxe,Melee,5,gp,simple,{'1hnd': {'slashing': {1: 6}}},"['Light', 'thrown (20/60)']",20,60,['STR'],2.0
4,javelin,Melee,5,sp,simple,{'1hnd': {'piercing': {1: 6}}},['Thrown (30/120)'],5,5,['STR'],2.0


Index(['Name', 'type', 'cost', 'cost_unit', 'main_type', 'dmg', 'props',
       'normal_range', 'long_range', 'mod', 'wgt'],
      dtype='object')

In [120]:
weps

,Name,type,cost,cost_unit,main_type,dmg,props,normal_range,long_range,mod,wgt
0,club,Melee,1,sp,simple,{'1hnd': {'bludgeoning': {1: 4}}},['Light'],5,5,['STR'],2.00
1,dagger,Melee,2,gp,simple,{'1hnd': {'piercing': {1: 4}}},"['Finesse', 'light', 'thrown (20/60)']",20,60,"['STR', 'DEX']",1.00
2,greatclub,Melee,2,sp,simple,{'1hnd': {'bludgeoning': {1: 8}}},['Two-handed'],5,5,['STR'],10.00
3,handaxe,Melee,5,gp,simple,{'1hnd': {'slashing': {1: 6}}},"['Light', 'thrown (20/60)']",20,60,['STR'],2.00
4,javelin,Melee,5,sp,simple,{'1hnd': {'piercing': {1: 6}}},['Thrown (30/120)'],5,5,['STR'],2.00
5,light hammer,Melee,2,gp,simple,{'1hnd': {'bludgeoning': {1: 4}}},"['Light', 'thrown (20/60)']",20,60,['STR'],2.00
6,mace,Melee,5,gp,simple,{'1hnd': {'bludgeoning': {1: 6}}},[],5,5,['STR'],4.00
7,quarterstaff,Melee,2,sp,simple,"{'1hnd': {'bludgeoning': {1: 6}}, '2hnd': {'bl...",['Versatile (1d8)'],5,5,['STR'],4.00
8,sickle,Melee,1,gp,simple,{'1hnd': {'slashing': {1: 4}}},['Light'],5,5,['STR'],2.00
9,spear,Melee,1,gp,simple,"{'1hnd': {'piercing': {1: 6}}, '2hnd': {'pierc...","['Thrown (20/60)', 'versatile (1d8)']",5,5,['STR'],3.00


In [121]:
for i,row in weps.iterrows():
    query = """INSERT INTO WEAPONS (name,type,cost,cost_unit,dmg,props,normal_range,long_range,mod,weight,subtype)
VALUES ('{}','{}',{},'{}',{},{},{},{},{},{},'{}')
"""
    t = query.format(
        row['Name'].lower(),row['type'],row['cost'],row['cost_unit'],row['dmg'],
        row['props'],row['normal_range'],row['long_range'],row['mod'],
        row['wgt'],row['main_type']
    )
    print(t)
    session.execute(t)


INSERT INTO WEAPONS (name,type,cost,cost_unit,dmg,props,normal_range,long_range,mod,weight,subtype)
VALUES ('club','Melee',1,'sp',{'1hnd': {'bludgeoning': {1: 4}}},['Light'],5,5,['STR'],2.0,'simple')

INSERT INTO WEAPONS (name,type,cost,cost_unit,dmg,props,normal_range,long_range,mod,weight,subtype)
VALUES ('dagger','Melee',2,'gp',{'1hnd': {'piercing': {1: 4}}},['Finesse', 'light', 'thrown (20/60)'],20,60,['STR', 'DEX'],1.0,'simple')

INSERT INTO WEAPONS (name,type,cost,cost_unit,dmg,props,normal_range,long_range,mod,weight,subtype)
VALUES ('greatclub','Melee',2,'sp',{'1hnd': {'bludgeoning': {1: 8}}},['Two-handed'],5,5,['STR'],10.0,'simple')

INSERT INTO WEAPONS (name,type,cost,cost_unit,dmg,props,normal_range,long_range,mod,weight,subtype)
VALUES ('handaxe','Melee',5,'gp',{'1hnd': {'slashing': {1: 6}}},['Light', 'thrown (20/60)'],20,60,['STR'],2.0,'simple')

INSERT INTO WEAPONS (name,type,cost,cost_unit,dmg,props,normal_range,long_range,mod,weight,subtype)
VALUES ('javelin','Melee',5,

In [76]:
session.execute('use dndio;')
for i, row in arm.iterrows():
    # print(row)
    query = """ 
INSERT INTO armor (
    baseac,
    cost_amt,
    cost_unit,
    doff,
    don,
    minstr,
    modac,
    name,
    stealth_disadv,
    type,
    weight
) VALUES (
    {},
    {},
    '{}',
    '{}',
    '{}',
    {},
    '{}',
    '{}',
    {},
    '{}',
    {}
);
""".format(
    row['baseAC'],row['cost_amt'],row['cost_unit'],row['Doff'],row['Don'],
    row['minSTR'],row['modAC'],row['Name'],str(row['stealth_disadv']).lower(),row['type'],row['Weight']
)
    print(query)
    session.execute(query)

 
INSERT INTO armor (
    baseac,
    cost_amt,
    cost_unit,
    doff,
    don,
    minstr,
    modac,
    name,
    stealth_disadv,
    type,
    weight
) VALUES (
    11,
    5,
    'gp',
    '1 minute',
    '1 minute',
    0,
    'DEX',
    'Padded',
    true,
    'light',
    8
);

 
INSERT INTO armor (
    baseac,
    cost_amt,
    cost_unit,
    doff,
    don,
    minstr,
    modac,
    name,
    stealth_disadv,
    type,
    weight
) VALUES (
    11,
    10,
    'gp',
    '1 minute',
    '1 minute',
    0,
    'DEX',
    'Leather',
    false,
    'light',
    10
);

 
INSERT INTO armor (
    baseac,
    cost_amt,
    cost_unit,
    doff,
    don,
    minstr,
    modac,
    name,
    stealth_disadv,
    type,
    weight
) VALUES (
    12,
    45,
    'gp',
    '1 minute',
    '1 minute',
    0,
    'DEX',
    'Studded Leather',
    false,
    'light',
    13
);

 
INSERT INTO armor (
    baseac,
    cost_amt,
    cost_unit,
    doff,
    don,
    minstr,
    modac,
    name,
  

In [6]:
skills
session.execute('use dndio;')
for i, row in skills.iterrows():
    query = """INSERT INTO skills (skill,modifier) VALUES ('{}','{}');""".format(row['skill'].lower(),row['modifier'])
    session.execute(query)

In [12]:
class_start = pd.read_csv('../../data/class_start_data.csv')

In [13]:
class_start.loc[12]

class                                                       Wizard
hp               {'hit_dice': {'1': 6}, 'start_hp': '6+CON', 'l...
equip            ['(a) a quarterstaff or (b) a dagger', '(a) a ...
proficiencies    {'Armor': ['none'], 'Weapons': ['dagger', 'dar...
spell_dc                                                8+PROF+INT
spell_atk_mod                                             PROF+INT
Name: 12, dtype: object

In [14]:
class_start

,class,hp,equip,proficiencies,spell_dc,spell_atk_mod
0,Artificer,"{'hit_dice': {'1': 8}, 'start_hp': '8+CON', 'l...","['any two simple weapons', 'a light crossbow a...","{'Armor': ['light', 'medium', 'shields'], 'Wea...",8+PROF+INT,PROF+INT
1,Barbarian,"{'hit_dice': {'1': 12}, 'start_hp': '12+CON', ...",['(a) a greataxe or (b) any martial melee weap...,"{'Armor': ['light', 'medium', 'shields'], 'Wea...",NaN,NaN
2,Bard,"{'hit_dice': {'1': 8}, 'start_hp': '8+CON', 'l...","['(a) a rapier, (b) a longsword, or (c) any si...","{'Armor': ['light'], 'Weapons': ['hand crossbo...",8+PROF+CHA,PROF+CHA
3,Cleric,"{'hit_dice': {'1': 8}, 'start_hp': '8+CON', 'l...",['(a) a mace or (b) a warhammer (if proficient...,"{'Armor': ['light', 'medium', 'shields'], 'Wea...",8+PROF+WIS,PROF+WIS
4,Druid,"{'hit_dice': {'1': 8}, 'start_hp': '8+CON', 'l...",[],"{'Armor': ['light', 'medium', 'shields (druids...",8+PROF+WIS,PROF+WIS
5,Fighter,"{'hit_dice': {'1': 10}, 'start_hp': '10+CON', ...","['(a) chain mail or (b) leather, longbow, and ...","{'Armor': ['light', 'medium', 'heavy', 'shield...",NaN,NaN
6,Monk,"{'hit_dice': {'1': 8}, 'start_hp': '8+CON', 'l...","['(a) a shortsword or (b) any simple weapon', ...","{'Armor': ['none'], 'Weapons': ['shortsword'],...",NaN,NaN
7,Paladin,"{'hit_dice': {'1': 10}, 'start_hp': '10+CON', ...",['(a) a martial weapon and a shield or (b) two...,"{'Armor': ['light', 'medium', 'heavy', 'shield...",8+PROF+CHA,PROF+CHA
8,Ranger,"{'hit_dice': {'1': 10}, 'start_hp': '10+CON', ...","['(a) scale mail or (b) leather armor', '(a) t...","{'Armor': ['light', 'medium', 'shields'], 'Wea...",8+PROF+WIS,PROF+WIS
9,Rogue,"{'hit_dice': {'1': 8}, 'start_hp': '8+CON', 'l...","['(a) a rapier or (b) a shortsword', '(a) a sh...","{'Armor': ['light'], 'Weapons': ['hand crossbo...",NaN,NaN


In [15]:
class_start['hp'][0]
# import json
# json.loads(class_start)

"{'hit_dice': {'1': 8}, 'start_hp': '8+CON', 'lvl_hp': {'1': 8}, 'alt_lvl_hp': '5+CON'}"

In [16]:
session.execute('use dndio;')
for i,row in class_start.iterrows():
    row_hp = json.loads(row['hp'].replace("'",'"'))
    # print(row_hp)
    hit_dice = row_hp['hit_dice']
    hit_dice = {int(k):v for k,v in hit_dice.items()}
    lvl_hp = row_hp['lvl_hp']
    lvl_hp = {int(k):v for k,v in lvl_hp.items()}
    profs = eval(row['proficiencies'])
    print(profs)
#     profs = json.loads(row['proficiencies'].replace("'","'"))
    query = """INSERT INTO class_start (char_class,hit_dice,lvl_hp,equip,proficiencies,start_hp,alt_lvl_hp,spell_dc,spell_atk_mod)
VALUES ('{}',{},{},{},{},'{}','{}','{}','{}')""".format(
    row['class'].lower(),hit_dice,lvl_hp,row['equip'],profs,row_hp['start_hp'],row_hp['alt_lvl_hp'],row['spell_dc'],row['spell_atk_mod']
)
    query = query.replace('"',"'")
    print(query)
    session.execute(query)


{'Armor': ['light', 'medium', 'shields'], 'Weapons': [], 'Tools': ["thieves'' tools", "tinker''s tools", "one type of artisan''s tools of your choice"], 'Saving Throws': ['CON', 'INT'], 'Skills': ['history', 'investigation', 'medicine', 'nature', 'perception', 'sleight of hand'], 'weapon_class': ['simple']}
INSERT INTO class_start (char_class,hit_dice,lvl_hp,equip,proficiencies,start_hp,alt_lvl_hp,spell_dc,spell_atk_mod)
VALUES ('artificer',{1: 8},{1: 8},['any two simple weapons', 'a light crossbow and 20 bolts', '(a) studded leather armor or (b) scale mail', 'thieves'' tools and a dungeoneer''s pack'],{'Armor': ['light', 'medium', 'shields'], 'Weapons': [], 'Tools': ['thieves'' tools', 'tinker''s tools', 'one type of artisan''s tools of your choice'], 'Saving Throws': ['CON', 'INT'], 'Skills': ['history', 'investigation', 'medicine', 'nature', 'perception', 'sleight of hand'], 'weapon_class': ['simple']},'8+CON','5+CON','8+PROF+INT','PROF+INT')
{'Armor': ['light', 'medium', 'shields']